In [1]:
import pandas as pd
import sklearn as skl
from sklearn.neighbors import NearestNeighbors
import scipy
from scipy.spatial import distance
import numpy
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('14_11_10_Green_20to21min.csv')
data

,Unnamed: 0,Unnamed: 0.1,source_file,frame_num,person_id,keypoint1_x,keypoint1_y,keypoint1_conf,keypoint2_x,keypoint2_y,...,keypoint22_conf,keypoint23_x,keypoint23_y,keypoint23_conf,keypoint24_x,keypoint24_y,keypoint24_conf,keypoint25_x,keypoint25_y,keypoint25_conf
0,799662,799662,Teacher_14010401_14_11_10_Original_Files_ZoomQ...,36000,-1,1127.7800,721.728,0.877695,1083.650,780.396,...,0.000000,0.000,0.00,0.00000,0.000,0.00,0.000000,0.000,0.0,0.00000
1,799663,799663,Teacher_14010401_14_11_10_Original_Files_ZoomQ...,36000,-1,44.6567,677.403,0.840910,0.000,0.000,...,0.000000,0.000,0.00,0.00000,0.000,0.00,0.000000,0.000,0.0,0.00000
2,799664,799664,Teacher_14010401_14_11_10_Original_Files_ZoomQ...,36000,-1,603.9880,677.432,0.846940,600.887,712.665,...,0.000000,0.000,0.00,0.00000,0.000,0.00,0.000000,0.000,0.0,0.00000
3,799665,799665,Teacher_14010401_14_11_10_Original_Files_ZoomQ...,36000,-1,480.2200,762.866,0.962323,453.769,759.812,...,0.053733,492.045,1074.82,0.12564,483.149,1074.84,0.153159,489.049,1051.3,0.16922
4,799666,799666,Teacher_14010401_14_11_10_Original_Files_ZoomQ...,36000,-1,512.6760,644.945,0.876525,524.307,645.110,...,0.000000,0.000,0.00,0.00000,0.000,0.00,0.000000,0.000,0.0,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41598,841260,841260,Teacher_14010401_14_11_10_Original_Files_ZoomQ...,37800,-1,789.3270,600.872,0.819472,789.192,621.421,...,0.000000,0.000,0.00,0.00000,0.000,0.00,0.000000,0.000,0.0,0.00000
41599,841261,841261,Teacher_14010401_14_11_10_Original_Files_ZoomQ...,37800,-1,798.1190,645.194,0.798595,795.209,674.560,...,0.000000,0.000,0.00,0.00000,0.000,0.00,0.000000,0.000,0.0,0.00000
41600,841262,841262,Teacher_14010401_14_11_10_Original_Files_ZoomQ...,37800,-1,1071.6700,648.213,0.399413,0.000,0.000,...,0.000000,0.000,0.00,0.00000,0.000,0.00,0.000000,0.000,0.0,0.00000
41601,841263,841263,Teacher_14010401_14_11_10_Original_Files_ZoomQ...,37800,-1,0.0000,0.000,0.000000,1404.440,742.108,...,0.000000,0.000,0.00,0.00000,0.000,0.00,0.000000,0.000,0.0,0.00000


In [3]:
# Plot of euclidian distances for all points in data
# frame_person_keypoints_dict[current_frame][person_id] = list of coordinates
frame_person_keypoints_dict = {}
for current_frame in data.frame_num.unique():
    frame_person_keypoints_dict[current_frame] = {}
    person_id = 0
    for i,person in data[data.frame_num==current_frame].iterrows():
        starter = 5
        count = 0
        person_25coordinates = numpy.zeros(shape=(25,2))
        while count<25:
            person_25coordinates[count]=person.iloc[starter:starter+2]
            starter = starter + 3
            count = count + 1
        frame_person_keypoints_dict[current_frame][person_id] = person_25coordinates
        person_id = person_id + 1

# Rule of thumb can be 1% of the video height = 19.2 distance
# Rule of thumb can be 1% of the video width = 10.8 distance
# Height of someone's head and that can be the effective rule of thumb
# Ambiguity distance

In [ ]:
# Calculate all euclidian distances between respective keypoints in the following frame
# SHOULD BE REWRITTEN TO USE 'groupby' INSTEAD

all_dists = []
for frame in data.frame_num.unique(): # To keep increasing order of frames
    for person in frame_person_keypoints_dict[frame].keys(): #for each person in the frame
        person_dists = [] #create a list for the person's distances
        for numpyarray_x_y in frame_person_keypoints_dict[frame][person]:
            idx = 0
            for person_next_frame in frame_person_keypoints_dict[frame+1].keys(): #for each person in the next frame
                dst = distance.euclidean(numpyarray_x_y, frame_person_keypoints_dict[frame+1][person_next_frame][idx])
                person_dists.append(dst)
            idx = idx + 1
    all_dists.extend(person_dists)

In [6]:
#pd.Series(all_dists).plot.density()
#ax.set_title('Overall Distances')
#plt.savefig('overall_distances.svg')

#dropped0s = list(filter((0.0).__ne__, all_dists))
#numpy.mean(dropped0s)

997.9593237730171